In [15]:
import pandas as pd
from datetime import timedelta
from datetime import datetime as dt
import yfinance as yf
import numpy as np

### Helper Functions

In [16]:
def get_next_friday():
    today = dt.today()
    offset = (4 - today.weekday()) % 7
    next_friday = today + timedelta(days=offset)
    return next_friday.strftime("%Y-%m-%d")

In [17]:
# Get the next next friday in YYYY-MM-DD format
def get_next_next_friday():
    today = datetime.today()
    offset = (4 - today.weekday()) % 7
    next_next_friday = today + timedelta(days=offset + 7)
    return next_next_friday.strftime("%Y-%m-%d")

### Assuming Profits

In [18]:
import datetime

# Initial setup
amount = 604.66
interest_rate = 1/100
current_date = datetime.date.today()
years = 1 + 1  # 1 for the current year

# Helper function to get the number of weeks in a month
def weeks_in_given_month(year, month):
    if month == 12:  # December
        next_month_first_day = datetime.date(year + 1, 1, 1)
    else:
        next_month_first_day = datetime.date(year, month + 1, 1)
    last_day_of_month = next_month_first_day - datetime.timedelta(days=1)
    first_day_of_month = datetime.date(year, month, 1)
    return (last_day_of_month - first_day_of_month).days // 7

# Function to calculate weekly earnings and reinvest
def weekly_earnings_and_reinvest_updated(amount, weeks):
    not_invested = 0  # Amount that's not invested
    
    for _ in range(weeks):
        interest_earned = (amount // 100) * interest_rate * 100
        total_amount = amount + interest_earned + not_invested
        
        # Determine how much can be reinvested
        reinvest = (total_amount // 100) * 100
        not_invested = total_amount - reinvest
        
        amount = reinvest
        
    return amount + not_invested

# Dictionary to store end-of-month amounts for each year
end_of_month_amounts = {}

# Calculate for the rest of the months of this year
current_month = current_date.month
amounts_for_current_year = []
for month in range(current_month, 13):  # Until December
    weeks = weeks_in_given_month(current_date.year, month)
    amount = weekly_earnings_and_reinvest_updated(amount, weeks)
    amounts_for_current_year.append(amount)
end_of_month_amounts[current_date.year] = amounts_for_current_year

# Calculate for each month of the next __ years
for year in range(1, years):
    amounts_for_year = []
    for month in range(1, 13):  # January to December
        weeks = weeks_in_given_month(current_date.year + year, month)
        amount = weekly_earnings_and_reinvest_updated(amount, weeks)
        amounts_for_year.append(amount)
    end_of_month_amounts[current_date.year + year] = amounts_for_year

# Formatting the output
month_names = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
output_string = ""

for year, amounts in end_of_month_amounts.items():
    output_string += f"{year}:\n"
    starting_month = current_month if year == current_date.year else 1
    for i, amount in enumerate(amounts):
        output_string += f"\t{month_names[starting_month - 1 + i]}: ${amount:,.2f}\n"

print(output_string)


2023:
	August: $628.66
	September: $652.66
	October: $676.66
	November: $700.66
	December: $728.66
2024:
	January: $756.66
	February: $784.66
	March: $813.66
	April: $845.66
	May: $877.66
	June: $910.66
	July: $946.66
	August: $982.66
	September: $1,020.66
	October: $1,060.66
	November: $1,100.66
	December: $1,144.66



# Viewing option chains

In [19]:
print(get_next_friday())

2023-09-01


In [20]:
file_viewer = pd.read_csv('naked_puts_results.csv')
filtered_data = file_viewer[file_viewer['Expiration Date'] == get_next_friday()].head(151)

# Display all rows, no limit
pd.set_option('display.max_rows', None)
# Display all columns, no limit
pd.set_option('display.max_columns', None)

filtered_data['ROI (%)'] = filtered_data['ROI (%)'].astype(float)
# filter out rows with ROI (%) greater than 0.8
filtered_data = filtered_data[filtered_data['ROI (%)'] > 0.8]
# filter out rows with strike price less than 6
filtered_data = filtered_data[filtered_data['strike'] < 6]

# sort by ROI (%) in ascending order
filtered_data = filtered_data.sort_values(by=['ROI (%)'], ascending=True)

# Show only the columns 'Expiration Date', 'ETF', 'Stock Price', 'bid', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'highPrice', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', and '52WeekLow'
filtered_data = filtered_data[['Expiration Date', 'ETF', 'Stock Price', 'bid', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'highPrice', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', '52WeekLow']]

filtered_data

,Expiration Date,ETF,Stock Price,bid,strike,ROI (%),OTM (%),volume,openInterest,highPrice,Original Stock Price,Original ROI (%),Original OTM (%),50-day MA,100-day MA,200-day MA,RSI,MACD,VWAP,52WeekHigh,52WeekLow
59,2023-09-01,KGC,5.04,0.04,5.0,0.81,0.79,2.0,794.0,0.09,5.01,1.63,0.20,4.85,4.95,4.59,49.55,-0.01,4.29,5.55,3.11
113,2023-09-01,SPCE,2.72,0.02,2.5,0.81,8.09,289.0,1090.0,0.04,2.59,1.63,3.47,3.75,3.78,4.30,14.29,-0.34,4.46,6.37,2.54
11,2023-09-01,APLD,6.28,0.05,5.5,0.92,12.42,60.0,98.0,0.10,5.82,1.85,5.50,7.98,6.77,4.53,28.85,-0.64,6.89,10.24,1.51
128,2023-09-01,WISH,5.28,0.05,5.0,1.01,5.30,9.0,125.0,0.14,5.08,2.88,1.57,7.13,7.50,12.56,28.19,-0.73,14.20,39.00,4.86
115,2023-09-01,STEM,5.45,0.05,5.0,1.01,8.26,23.0,307.0,0.10,5.12,2.04,2.34,6.13,5.48,7.49,22.24,-0.38,8.09,17.42,3.80
19,2023-09-01,BLNK,4.16,0.04,4.0,1.01,3.85,331.0,528.0,0.08,4.07,2.04,1.72,5.88,6.48,8.91,21.87,-0.56,9.48,23.23,4.07
39,2023-09-01,HBI,5.06,0.05,5.0,1.01,1.19,17.0,838.0,0.05,5.07,1.01,1.38,4.93,4.74,5.55,35.82,0.04,6.01,9.47,3.87
125,2023-09-01,VERU,1.02,0.01,1.0,1.01,1.96,200.0,317.0,0.01,1.03,1.01,2.91,1.16,1.17,2.89,39.42,-0.04,6.97,16.49,0.98
22,2023-09-01,CLOV,1.23,0.01,1.0,1.01,18.70,60.0,936.0,0.01,1.23,1.01,18.70,1.12,1.01,1.06,42.90,0.04,1.21,2.75,0.73
122,2023-09-01,UCAR,4.55,0.05,4.0,1.27,12.09,NaN,3.0,0.05,4.55,1.27,12.09,6.07,NaN,NaN,43.41,-0.45,8.42,43.18,3.11


In [21]:
# Get the dataFrame from '/Data/putsDataSuccessFailed.csv'
df = pd.read_csv('Data/putsDataSuccessFailed.csv')

# Define your color function for entire row
def color_row(row):
    color = 'lightgreen' if row['Status'] == 'Success' else 'lightcoral'
    return ['background-color: %s' % color]*len(row)

# sort the rows by Original ROI(%) in ascending order
df = df.sort_values(by=['Original ROI (%)'], ascending=True)

# Apply the color function to the entire DataFrame
styled_df = df.style.apply(color_row, axis=1).format("{:.2f}", subset=df.select_dtypes(include=['float64']).columns)

# Display the styled DataFrame
styled_df

EmptyDataError: No columns to parse from file

In [ ]:
# Columns for Machine Learning
columns = ['strike', 'volume', 'openInterest', 'highPrice', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', '52WeekLow']